## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-23-02-37-38 +0000,world,Trump announces trade deal with Japan that wou...,https://www.cbc.ca/news/world/trump-japan-trad...
1,2025-07-22-22-50-27 +0000,world,South Australia algal bloom a 'natural disaste...,https://www.cbc.ca/news/climate/south-australi...
2,2025-07-22-21-22-45 +0000,world,Green energy has passed 'positive tipping poin...,https://www.cbc.ca/news/climate/green-energy-r...
3,2025-07-22-19-49-19 +0000,world,'Go f--k yourself!': What Stephen Colbert and ...,https://www.cbc.ca/news/entertainment/colbert-...
4,2025-07-22-18-56-32 +0000,world,"Ozzy Osbourne, godfather of heavy metal who fo...",https://www.cbc.ca/news/entertainment/ozzy-osb...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2385/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,south,2
35,late,2
59,crash,2
1,announces,1
0,trump,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-07-22-19-49-19 +0000,world,'Go f--k yourself!': What Stephen Colbert and ...,https://www.cbc.ca/news/entertainment/colbert-...,14
5,2025-07-22-18-14-33 +0000,world,South Korean pilot may have shut off less-dama...,https://www.cbc.ca/news/world/south-korea-angr...,13
1,2025-07-22-22-50-27 +0000,world,South Australia algal bloom a 'natural disaste...,https://www.cbc.ca/news/climate/south-australi...,11
2,2025-07-22-21-22-45 +0000,world,Green energy has passed 'positive tipping poin...,https://www.cbc.ca/news/climate/green-energy-r...,10
0,2025-07-23-02-37-38 +0000,world,Trump announces trade deal with Japan that wou...,https://www.cbc.ca/news/world/trump-japan-trad...,10


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,14,2025-07-22-19-49-19 +0000,world,'Go f--k yourself!': What Stephen Colbert and ...,https://www.cbc.ca/news/entertainment/colbert-...
5,13,2025-07-22-18-14-33 +0000,world,South Korean pilot may have shut off less-dama...,https://www.cbc.ca/news/world/south-korea-angr...
0,10,2025-07-23-02-37-38 +0000,world,Trump announces trade deal with Japan that wou...,https://www.cbc.ca/news/world/trump-japan-trad...
2,10,2025-07-22-21-22-45 +0000,world,Green energy has passed 'positive tipping poin...,https://www.cbc.ca/news/climate/green-energy-r...
4,10,2025-07-22-18-56-32 +0000,world,"Ozzy Osbourne, godfather of heavy metal who fo...",https://www.cbc.ca/news/entertainment/ozzy-osb...
1,9,2025-07-22-22-50-27 +0000,world,South Australia algal bloom a 'natural disaste...,https://www.cbc.ca/news/climate/south-australi...
6,8,2025-07-22-10-09-26 +0000,world,Angry students confront Bangladesh officials a...,https://www.cbc.ca/news/world/bangladesh-air-f...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
